## Simple UDF sample

This notebook shows how to apply [user defined functions](https://open-eo.github.io/openeo-python-client/udf.html).
This is a powerfull concept that allows you to combine arbitrary Python code with your predefined openEO processes.

We start by setting up a basic data cube:


In [1]:
import openeo

openeo.client_version()

'0.16.0a3'

In [2]:
# Create connection to openEO back-end
connection = openeo.connect("openeo.cloud").authenticate_oidc()

Authenticated using refresh token.


In [3]:
# Load initial data cube.
s2_cube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent={"west": 4.00, "south": 51.04, "east": 4.01, "north": 51.05},
    temporal_extent=["2022-03-01", "2022-03-31"],
    bands=["B02", "B03", "B04"],
)

Now we load the UDF, it needs to be provided to the backend as plain text, so we store it in a separate file next to this notebook. This makes it easier to edit.

In [4]:
simple_udf = openeo.UDF.from_file("simple_udf.py")

print(simple_udf.code)

from openeo.udf import XarrayDataCube

def apply_datacube(cube: XarrayDataCube, context: dict) -> XarrayDataCube:
    """
    A trivial UDF that scales values, can also be done without UDF!
    """
    array = cube.get_array()
    array.values = 0.0001 * array.values
    return cube


We can now apply the UDF to each pixel, using it as child process in the `apply` process:

In [5]:
rescaled = s2_cube.apply(simple_udf)

In [6]:
%%time
rescaled.download("apply-udf-scaling.nc")

CPU times: user 16.3 ms, sys: 8.44 ms, total: 24.8 ms
Wall time: 40.7 s
